In [8]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
from itertools import chain

## 6.3 ヒューリスティック探索
### 6.3.1 最良優先探索

In [24]:
connect = [[1, 5], [0, 2, 6], [1, 3, 7], [2, 4, 8], [3, 9],
        [6, 0, 10], [5, 7, 1, 11], [6, 8, 2, 12], [7, 9, 3, 13], [8, 4, 14],
        [11, 5, 15], [10, 12, 6, 16], [11, 13, 7, 17], [12, 14, 8, 18], [13, 9, 19],
        [16, 10, 20], [15, 17, 11, 21], [16, 18, 12, 22], [17, 19, 13, 23], [18, 14, 24],
        [21, 15], [20, 22, 16], [21, 23, 17], [22, 24, 18], [23, 19]]

width = 5
height = 5
goal = 23
hn = []

def manhattan_distance(i, height, width, goal):
    return np.abs(i//height - goal//height) + np.abs(i%width - goal%width)

# max(chain(*connect))はconnectの中の最大値を抜き出してるだけ.
for i in range(max(chain(*connect)) + 1):
    hn.append(manhattan_distance(i, height, width, goal))

prev = [-1]*(max(chain(*connect)) + 1)
open_list = [0]
close_list = []

while open_list:
    now = open_list.pop(0)
    close_list.append(now)
    a = [hn[i] for i in connect[now]]
    b = np.argsort(a)[0]
    c = connect[now][b]
    print(now, connect[now], a, b, c)
    prev[c] = now
    open_list.append(c)
    if c==goal:
        break


print(f'close_list:\n {close_list}')
print(f'prev:\n {prev}')

0 [1, 5] [6, 6] 0 1
1 [0, 2, 6] [7, 5, 5] 1 2
2 [1, 3, 7] [6, 4, 4] 1 3
3 [2, 4, 8] [5, 5, 3] 2 8
8 [7, 9, 3, 13] [4, 4, 4, 2] 3 13
13 [12, 14, 8, 18] [3, 3, 3, 1] 3 18
18 [17, 19, 13, 23] [2, 2, 2, 0] 3 23
close_list:
 [0, 1, 2, 3, 8, 13, 18]
prev:
 [-1, 0, 1, 2, -1, -1, -1, -1, 3, -1, -1, -1, -1, 8, -1, -1, -1, -1, 13, -1, -1, -1, -1, 18, -1]


In [28]:
for i in range(5):
    print(hn[i*5:i*5+5])

[7, 6, 5, 4, 5]
[6, 5, 4, 3, 4]
[5, 4, 3, 2, 3]
[4, 3, 2, 1, 2]
[3, 2, 1, 0, 1]
